In [64]:
import arcpy
from arcpy import env
import os
import numpy as np
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.pivot_table(df, values='a', index='b', columns='c', aggfunc='sum', fill_value=0)
# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [65]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [66]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = '.\\Outputs'

In [67]:
hui_df = pd.DataFrame.spatial.from_featureclass(r'.\Outputs\hui_for_web.gdb\hui_2022_pts_web_version')
hui_df.head()

,OBJECTID,UNIT_ID,TYPE,SUBTYPE,IS_OUG,UNIT_COUNT,DUA,ACRES,TOT_BD_FT2,TOT_VALUE,APX_BLT_YR,BLT_DECADE,CITY,COUNTY,SUBCOUNTY,BLT_YR2,CENTER,CENTERTYPE,DIST_FR,DIST_LR,DIST_BRT,DIST_FWYE,ORIG_FID,SHAPE
0,1,1.0,multi_family,apartment,0,2.0,6.17,0.323897,2448.0,377500.0,1956.0,1950,West Valley City,Salt Lake,North Salt Lake County,1956-01-01,None,None,4.19,0.54,0.44,0.87,1,"{""x"": 418290.5295000002, ""y"": 4504686.09909999..."
1,2,2.0,multi_family,apartment,0,7.0,76.03,0.092074,7996.0,2122800.0,1911.0,1910,Salt Lake City,Salt Lake,North Salt Lake County,1911-01-01,University Gardens,Neighborhood Center,2.68,0.02,0.27,2.98,2,"{""x"": 427910.70189999975, ""y"": 4513471.0704, ""..."
2,3,3.0,multi_family,duplex,0,2.0,14.82,0.134912,2551.0,675800.0,1909.0,1900,Salt Lake City,Salt Lake,North Salt Lake County,1909-01-01,None,None,2.45,0.48,1.08,1.75,3,"{""x"": 426668.074, ""y"": 4510722.966499999, ""spa..."
3,4,4.0,multi_family,duplex,0,2.0,14.53,0.137643,2004.0,772200.0,1906.0,1900,Salt Lake City,Salt Lake,North Salt Lake County,1906-01-01,None,None,2.66,0.25,1.32,1.80,4,"{""x"": 427069.81159999967, ""y"": 4510769.7737000..."
4,5,5.0,multi_family,apartment,0,5.0,31.96,0.156440,2688.0,722800.0,1963.0,1960,Salt Lake City,Salt Lake,North Salt Lake County,1963-01-01,None,None,1.51,0.56,0.68,1.83,5,"{""x"": 426016.8798000002, ""y"": 4514324.7586, ""s..."


In [68]:
#=========
# Base 
#=========

counties_all =  ["None"] + sorted(list(set(hui_df["COUNTY"].to_list())))
cities_all = ["None"] + sorted(list(set(hui_df['CITY'].to_list())))
centers_all =  ["None"] + sorted(list(set(hui_df['CENTER'].to_list())))


counties_text = str(counties_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "BASE.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

f.write("\t{\n")
f.write(f'''\t\t"COUNTY":{counties_text},\n''')
f.write(f'''\t\t"CITY":{cities_text},\n''')
f.write(f'''\t\t"CENTER":{centers_text}\n''')
f.write("\t}\n")


# close the json
f.write("]\n")
f.close()

In [69]:
#=========
# County 
#=========
# hui_df[hui_df['COUNTY'].isna()==False]
counties_all = ["None"] + sorted(list(set(hui_df[hui_df['COUNTY']!= 'None']["COUNTY"].to_list())))
cities_all = ["None"] + sorted(list(set(hui_df[hui_df['CITY']!= 'None']["CITY"].to_list())))
centers_all = ["None"] + sorted(list(set(hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list())))

# cities_all.remove('None')
# centers_all.remove('None')

counties_text = str(counties_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')
 
# create and open the json
json = os.path.join(outputs, "COUNTY.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")


for i in counties_all: 
    
    if i != "None":
        df = hui_df[hui_df['COUNTY'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df[df['CITY']!= 'None']['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df[df['CENTER']!= 'None']['CENTER'].to_list())))

        # counties_text = str(counties).replace("'", '"')
        cities_text = str(cities).replace("'", '"')
        centers_text = str(centers).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        if i == counties_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#=========
# City 
#=========

counties_all = ["None"] + sorted(list(set(hui_df[hui_df['COUNTY']!= 'None']["COUNTY"].to_list())))
cities_all = ["None"] + sorted(list(set(hui_df[hui_df['CITY']!= 'None']["CITY"].to_list())))
centers_all = ["None"] + sorted(list(set(hui_df[hui_df['CENTER']!= 'None']["CENTER"].to_list())))


counties_text = str(counties_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "CITY.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")



for i in cities_all: 
    if i != "None":
        df = hui_df[hui_df['CITY'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df[df['CENTER']!= 'None']["CENTER"].to_list())))


        counties_text = str(counties).replace("'", '"')
        centers_text = str(centers).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        if i == cities_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#=========
# Center 
#=========

counties_all = ["None"] + sorted(list(set(hui_df["COUNTY"].to_list())))
cities_all = ["None"] + sorted(list(set(hui_df["CITY"].to_list())))
centers_all = ["None"] + sorted(list(set(hui_df["CENTER"].to_list())))

counties_text = str(counties_all).replace("'", '"')
cities_text = str(cities_all).replace("'", '"')
centers_text = str(centers_all).replace("'", '"')

# create and open the json
json = os.path.join(outputs, "CENTER.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

for i in centers_all: 
    
    if i != "None":
        df = hui_df[hui_df['CENTER'] == i].copy()

        counties = ["None"] + sorted(list(set(df[df['COUNTY']!= 'None']['COUNTY'].to_list())))
        cities = ["None"] + sorted(list(set(df[df['CITY']!= 'None']['CITY'].to_list())))
        centers = ["None"] + sorted(list(set(df['CENTER'].to_list())))

        counties_text = str(counties).replace("'", '"')
        cities_text = str(cities).replace("'", '"')

        f.write("\t{\n")
        f.write(f'''\t\t"NAME":"{i}",\n''')
        f.write(f'''\t\t"COUNTY":{counties_text},\n''')
        f.write(f'''\t\t"CITY":{cities_text},\n''')
        f.write(f'''\t\t"CENTER":{centers_text}\n''')
        
        if i == centers_all[-1]: f.write("\t}\n")
        else: f.write("\t},\n")


# close the json
f.write("]\n")
f.close()

In [ ]:
#=========
# TYPE
#==========

types = sorted(list(set(hui_df['TYPE'].to_list())),reverse=True)
subtypes = sorted(list(set(hui_df['SUBTYPE'].to_list())))

types_text = str(types).replace("'", '"')
subtypes_text = str(subtypes).replace("'", '"')

json = os.path.join(outputs, "TYPE.json")
if os.path.exists(json):
    os.remove(json)

f = open(json, "a")
f.write("[\n")

f.write("\t{\n")
f.write(f'''\t\t"TYPE":{types_text},\n''')
f.write(f'''\t\t"SUBTYPE":{subtypes_text}\n''')
f.write("\t}\n")

# close the json
f.write("]\n")
f.close()